# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **Guelph, Ontario, Canada**, or **Canada** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Guelph, Ontario, Canada** to Ann Arbor, USA. In that case at least one source file must be about **Guelph, Ontario, Canada**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Guelph, Ontario, Canada** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wdgts
import inspect
from scipy.stats.stats import pearsonr

storm = pd.read_csv('Storm_Regular_Season.csv')
knights = pd.read_csv('Knights_Regular_Season.csv')

# Note: OTL = Overtime Loss; SL = Shootout Loss
print(storm.head())
print(knights.head())

    Season   Games   Won   Lost  Tied  OTL  SL   Points   Pct %   Goals For  \
0  1991–92       66     4     51    11   -   -        19   0.144        235   
1  1992–93       66    27     33     6   -   -        60   0.455        298   
2  1993–94       66    32     28     6   -   -        70   0.530        323   
3  1994–95       66    47     14     5   -   -        99   0.750        330   
4  1995–96       66    45     16     5   -   -        95   0.720        297   

   Goals Against     Standing  
0            425     8th Emms  
1            360     7th Emms  
2            290     3rd Emms  
3            200  1st Central  
4            186  1st Central  
    Season   Games   Won   Lost  Tied  OTL  SL   Points   Pct %   Goals for  \
0  1965–66       48    12     29     7   -   -        31   0.323        149   
1  1966–67       48    18     21     9   -   -        45   0.469        185   
2  1967–68       54    17     31     6   -   -        40   0.370        177   
3  1968–69       

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

storm.columns = storm.columns.str.replace(pat='[%\$\+\=]', repl='') \
    .str.replace(pat='^ +', repl='') \
    .str.replace(pat=' +$', repl='') \
    .str.replace(pat=' ', repl='_') \
    .str.upper();

knights.columns = knights.columns.str.replace(pat='[%\$\+\=]', repl='') \
    .str.replace(pat='^ +', repl='') \
    .str.replace(pat=' +$', repl='') \
    .str.replace(pat=' ', repl='_') \
    .str.upper();

warnings.simplefilter(action='always', category=FutureWarning)

In [3]:
# Make missing columns numeric and fill them with 0

storm['TIED'] = storm['TIED'].apply(pd.to_numeric, errors = 'coerce')
storm['OTL'] = storm['OTL'].apply(pd.to_numeric, errors = 'coerce')
storm['SL'] = storm['SL'].apply(pd.to_numeric, errors = 'coerce')
storm = storm.fillna(0)

knights['TIED'] = knights['TIED'].apply(pd.to_numeric, errors = 'coerce')
knights['OTL'] = knights['OTL'].apply(pd.to_numeric, errors = 'coerce')
knights['SL'] = knights['SL'].apply(pd.to_numeric, errors = 'coerce')
knights = knights.fillna(0)

# Include Overtime & Shootout losses in column LOST

storm['LOST'] = storm['LOST'] + storm['OTL'] + storm['SL']
storm['TIED'] = storm['GAMES'] - storm['WON'] - storm['LOST']

knights['LOST'] = knights['LOST'] + knights['OTL'] + knights['SL']
knights['TIED'] = knights['GAMES'] - knights['WON'] - knights['LOST']

# Drop Overtime & Shootout Loss columns, and Standing column

storm.drop(['OTL', 'SL', 'STANDING'], axis = 1, inplace = True)
knights.drop(['OTL', 'SL', 'STANDING'], axis = 1, inplace = True)


In [4]:
storm.columns = [col + '_S' for col in storm.columns]
storm = storm.rename(columns={'SEASON_S':'SEASON'})

knights.columns = [col + '_K' for col in knights.columns]
knights = knights.rename(columns={'SEASON_K':'SEASON'})

hockey = pd.merge(storm, knights, how = 'inner', left_on = 'SEASON', right_on = 'SEASON')
hockey.set_index('SEASON')

,GAMES_S,WON_S,LOST_S,TIED_S,POINTS_S,PCT_S,GOALS_FOR_S,GOALS_AGAINST_S,GAMES_K,WON_K,LOST_K,TIED_K,POINTS_K,PCT_K,GOALS_FOR_K,GOALS_AGAINST_K,PLAYOFFS_K
SEASON,,,,,,,,,,,,,,,,,
1991–92,66,4,51.0,11.0,19,0.144,235,425,66,37,25.0,4.0,78,0.591,310,260,Lost in Quarterfinals
1992–93,66,27,33.0,6.0,60,0.455,298,360,66,32,27.0,7.0,71,0.538,323,292,Lost in Quarterfinals
1993–94,66,32,28.0,6.0,70,0.530,323,290,66,32,30.0,4.0,68,0.515,293,279,Lost in Quarterfinals
1994–95,66,47,14.0,5.0,99,0.750,330,200,66,18,44.0,4.0,40,0.303,210,309,Lost in Quarterfinals
1995–96,66,45,16.0,5.0,95,0.720,297,186,66,3,60.0,3.0,9,0.068,179,435,Missed Playoffs
1996–97,66,35,25.0,6.0,78,0.591,300,251,66,13,51.0,2.0,28,0.212,215,365,Missed Playoffs
1997–98,66,42,18.0,6.0,90,0.682,263,189,66,40,21.0,5.0,85,0.644,301,238,Lost in Conference Final
1998–99,68,44,22.0,2.0,90,0.662,300,218,68,34,30.0,4.0,72,0.529,260,217,Lost OHL Championship
1999–2000,68,29,35.0,4.0,63,0.463,250,256,68,22,39.0,7.0,54,0.397,186,250,Missed Playoffs


In [5]:
# Remove any seasons in which either team did not participate

hockey = hockey[hockey.GAMES_S != 0]
hockey = hockey[hockey.GAMES_K != 0]

In [54]:
def onChange(x):
    
    fig = plt.figure(figsize = (20,10))
    
    marker_S = '^'
    marker_K = 'o'
    markersize = 12
    linestyle_S = ':' 
    linestyle_K = '--' 
        
    
    if x == 'Wins':
        ax = hockey.WON_S.plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                               label = "Storm Wins", linewidth = 3, color = 'blue')
        ax = hockey.WON_K.plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                               label = "Kinghts Wins", linewidth = 3, color = 'green')   
        corr, pvalue = pearsonr(hockey.WON_S, hockey.WON_K)
    elif x == 'Losses':
        ax = hockey.LOST_S.plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                                label = "Storm Losses", linewidth = 3, color = 'blue')
        ax = hockey.LOST_K.plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                                label = "Kinghts Losses", linewidth = 3, color = 'green')
        corr, pvalue = pearsonr(hockey.LOST_S, hockey.LOST_K)
    elif x == 'Ties':
        ax = hockey.TIED_S[0:14].plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                                      label = "Storm Ties", linewidth = 3, color = 'blue')
        ax = hockey.TIED_K[0:14].plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                                      label = "Kinghts Ties", linewidth = 3, color = 'green')
        corr, pvalue = pearsonr(hockey.TIED_S[0:14], hockey.TIED_K[0:14])
    elif x == 'Goals For':
        ax = hockey.GOALS_FOR_S.plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                                     label = "Storm - Goals For", linewidth = 3, color = 'blue')
        ax = hockey.GOALS_FOR_K.plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                                     label = "Kinghts - Goals For", linewidth = 3, color = 'green')
        corr, pvalue = pearsonr(hockey.GOALS_FOR_S, hockey.GOALS_FOR_K)
    elif x == 'Goals Against':
        ax = hockey.GOALS_AGAINST_S.plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                                         label = "Storm - Goals Against", linewidth = 3, color = 'blue')
        ax = hockey.GOALS_AGAINST_K.plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                                         label = "Kinghts - Goals Against", linewidth = 3, color = 'green')
        corr, pvalue = pearsonr(hockey.GOALS_AGAINST_S, hockey.GOALS_AGAINST_K)
    elif x == 'Points':
        ax = hockey.POINTS_S.plot(marker = marker_S, markersize = markersize, linestyle = linestyle_S, 
                                  label = "Storm Points", linewidth = 3, color = 'blue')
        ax = hockey.POINTS_K.plot(marker = marker_K, markersize = markersize, linestyle = linestyle_K, 
                                  label = "Kinghts Points", linewidth = 3, color = 'green')
        corr, pvalue = pearsonr(hockey.POINTS_S, hockey.POINTS_K)
        
    
    ax.set_xlabel('Season', fontsize = 24, fontweight = 'bold')    
    ax.set_ylabel(x, fontsize = 24, fontweight = 'bold')
    
    if x != 'Ties':
        ax.set_xticks(list(range(0, hockey.shape[0])))
        ax.set_xticklabels(list(hockey['SEASON']))
    else:
        ax.set_xticks(list(range(0, hockey[0:14].shapprese[0])))
        ax.set_xticklabels(list(hockey[0:14]['SEASON']))
                
    ax.tick_params(axis = 'x', labelsize = 14, rotation = 80)
    ax.tick_params(axis = 'y', labelsize = 14)    
    
    
    plt.title('\nComparison Of OHL Junior Hockey Teams:\n Guelph Storm vs. London Knights\n',
             fontsize = 28, fontweight = 'bold')
        
    plt.legend(prop = {'size': 16})    
    
    upperCorr = ['Ties', 'Losses', 'Goals Against']
    lowerCorr = ['Wins', 'Points', 'Goals For']
    
    if x in lowerCorr:
        plt.text(len(ax.get_xticks())//3, 
                 ax.get_yticks()[1] + 0.2 * (ax.get_yticks()[2] - ax.get_yticks()[1]),
                 "Corr = " + f"{corr:.4f}""; P-value = " + f"{pvalue:.4f}", size = 16)
        
    elif x in upperCorr: 
        print(ax.get_yticks()[-2] + 0.2 * (ax.get_yticks()[-2] - ax.get_yticks()[-3]))
        plt.text(len(ax.get_xticks())//3, 
                 ax.get_yticks()[-2] - 0.3 * (ax.get_yticks()[-2] - ax.get_yticks()[-3]),
                 "Corr = " + f"{corr:.4f}""; P-value = " + f"{pvalue:.4f}", size = 16)
        
# Configure dropdown 
opts = ['Wins', 'Losses', 'Ties', 'Goals For', 'Goals Against', 'Points']
wdgts.interact(onChange, x = opts);
      

interactive(children=(Dropdown(description='x', options=('Wins', 'Losses', 'Ties', 'Goals For', 'Goals Against…